# Convert from a fixed pattern on the bottom to a mapping for the positions

Suppose we have a fixed pattern with known dimensions on the bottom of the setup. This pattern is distorted due to refractions. We can create a mapping from the observed pattern by the camera back to the real coordinates.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as optimization
plt.rcParams['figure.dpi'] = 200

## Create a pattern and distort it

In [ ]:
# Lines (1D)

n_lines = 10
x_lines = np.linspace(-1,1,n_lines) 

# Distortion
mag_dist = 0.1
x_lines_dist = x_lines + mag_dist*(2*np.random.rand(n_lines)-1)

fig, ax = plt.subplots(1,1)
for i in range(0,n_lines,1):
    plt.axvline(x_lines[i],color='k')
    plt.axvline(x_lines_dist[i],color='k',linestyle=':')
plt.legend(['Real','Distorted'])
plt.title('Position of lines on the bottom')
plt.xlabel('x')
plt.ylabel('y')
ax.set_aspect('equal')
plt.show()


# Dots (2D)

n_x_dots = 10
n_y_dots = 5
x_dots, y_dots = np.meshgrid(np.linspace(-1,1,n_x_dots),np.linspace(-1,1,n_y_dots)) 
x_dots = x_dots.flatten()
y_dots = y_dots.flatten()

# Distortion
mag_dist = 0.05
x_dots_dist = x_dots + mag_dist*(2*np.random.rand(n_x_dots*n_y_dots)-1)
y_dots_dist = y_dots + mag_dist*(2*np.random.rand(n_x_dots*n_y_dots)-1)

fig, ax = plt.subplots(1,1)
plt.plot(x_dots,y_dots,'k.',label='Real')
plt.plot(x_dots_dist,y_dots_dist,'k.',mfc='w',label='Distorted')
plt.legend()
plt.title('Position of dots on the bottom')
plt.xlabel('x')
plt.ylabel('y')
ax.set_aspect('equal')
plt.show()

# Assign pixel values to distorted points and a surface through them
## 1D example

In [ ]:
# Construct actual bottom positions
n_x_dots = 10
x_dots_real = np.linspace(-1,1,n_x_dots)

# Come up with (fictional) corresponding pixel values
x_dots_pixel = np.asarray([int(np.sign(x)*abs(x)**0.9 * 1000 + np.random.rand()) for x in x_dots_real])


# Example functions to be used here
def surface_1D(x, A,B,C,D):
    return A*x**3 + B*x**2 + C*x + D


# 1D 
popt, pcov = optimization.curve_fit(surface_1D, x_dots_pixel, x_dots_real)
perr = np.sqrt(np.diag(pcov))


print(*popt)


plt.figure(figsize=(12,8))
plt.plot(x_dots_pixel,x_dots_real,'kx',label='Data')
xrange = np.linspace(min(x_dots_pixel),max(x_dots_pixel),100)
plt.plot(xrange,surface_1D(xrange,*popt),'r-',label='Best fit')
plt.xlabel('Image position [px]')
plt.ylabel('Real position [m]')
plt.legend()
plt.grid()
plt.show()

## 2D Example

In [ ]:
# Construct actual bottom positions
n_x_dots = 10
n_y_dots = 5
x_dots, y_dots = np.meshgrid(np.linspace(-1,1,n_x_dots),np.linspace(-1,1,n_y_dots)) 
x_dots_real = x_dots.flatten()
y_dots_real = y_dots.flatten()


# Come up with (fictional) corresponding pixel values, and distort them based on some common value
x_dots_pixel = x_dots_real * 1000 * np.sqrt(x_dots_real**2+y_dots_real**2)
y_dots_pixel = y_dots_real * 1000 * np.sqrt(x_dots_real**2+y_dots_real**2)

x_dots_pixel = [int(x) for x in x_dots_pixel]
y_dots_pixel = [int(y) for y in y_dots_pixel]

# Example functions to be used here


def surface_2D(xy, A,B,C,D,E,F):
    x = xy[0,:]
    y = xy[1,:]
    return A*x**2 + B*y**2 + C*x + D*y + E*x*y + F 

# 2D
poptx, pcov = optimization.curve_fit(surface_2D, np.vstack((x_dots_pixel,y_dots_pixel)), x_dots_real) # x deformation
popty, pcov = optimization.curve_fit(surface_2D, np.vstack((y_dots_pixel,x_dots_pixel)), y_dots_real) # y deformation

print(*poptx)
print(*popty)

# Show deformation on a regular grid
plt.figure(figsize=(12,8))
plt.plot(x_dots_real,y_dots_real,'kx',label='Real positions')
plt.plot(surface_2D(np.vstack((x_dots_pixel,y_dots_pixel)),*poptx), surface_2D(np.vstack((y_dots_pixel,x_dots_pixel)),*popty),'.r',label='Recovered positions')
plt.legend()
plt.grid()
plt.show()

plt.figure()
plt.plot(x_dots_pixel,x_dots_real,'kx',label='Real data')
plt.plot(x_dots_pixel,surface_2D(np.vstack((x_dots_pixel,y_dots_pixel)),*poptx),'.r',label='Recovered data')
plt.xlabel('Image position [px]')
plt.ylabel('Real position [m]')
plt.legend()
plt.grid()
plt.show()